<a href="https://colab.research.google.com/github/mtzig/LIDC_GDRO/blob/main/notebooks/LIDC_malig_subgroup.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%load_ext autoreload
%autoreload 2

In [136]:
# %cd ..
!git clone https://github.com/mtzig/LIDC_GDRO.git
%cd /content/LIDC_GDRO

Cloning into 'LIDC_GDRO'...
remote: Enumerating objects: 3538, done.
remote: Counting objects: 100% (110/110), done.
remote: Compressing objects: 100% (83/83), done.
remote: Total 3538 (delta 59), reused 62 (delta 27), pack-reused 3428
Receiving objects: 100% (3538/3538), 68.48 MiB | 30.41 MiB/s, done.
Resolving deltas: 100% (3207/3207), done.
Checking out files: 100% (5396/5396), done.
/content/LIDC_GDRO


In [140]:
# !git pull

remote: Enumerating objects: 14, done.
remote: Counting objects: 100% (14/14), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 8 (delta 6), reused 8 (delta 6), pack-reused 0
Unpacking objects: 100% (8/8), done.
From https://github.com/mtzig/LIDC_GDRO
   44d8b03..bf33e58  main       -> origin/main
Updating 44d8b03..bf33e58
Fast-forward
 data/LIDC_data_split.csv            | 1489 +++++++++++++++++++++++++++++++++++
 image_data_utils.py                 |   42 +-
 notebooks/LIDC_clusters.ipynb       |  104 ++-
 notebooks/LIDC_malig_subgroup.ipynb |   55 +-
 4 files changed, 1614 insertions(+), 76 deletions(-)
 create mode 100644 data/LIDC_data_split.csv


In [149]:
import pandas as pd
# import numpy as np
from dataloaders import InfiniteDataLoader
from datasets import NoduleDataset, SubclassedNoduleDataset
from models import TransferModel
from loss import ERMLoss, GDROLossAlt#, GDROLoss
from train import train, test
from torch.optim.lr_scheduler import ReduceLROnPlateau

# import torchvision
from torchvision import transforms
import torch
from image_data_utils import getImages

In [138]:
if torch.cuda.is_available():
    print("Good to go!")
    DEVICE = torch.device("cuda")
else:
    print("Using cpu")
    DEVICE = torch.device("cpu")

Good to go!


In [143]:
train_data, cv_data, test_data = getImages(split=True, sublabels=True, binary=True, device=DEVICE)

In [145]:
tr = SubclassedNoduleDataset(*train_data)
cv = SubclassedNoduleDataset(*cv_data)
tst = SubclassedNoduleDataset(*test_data)
tr_loader = InfiniteDataLoader(tr, batch_size=128)
cv_loader = InfiniteDataLoader(cv, len(cv))
tst_loader = InfiniteDataLoader(tst, len(tst))

In [159]:
model = TransferModel(device=DEVICE, pretrained=True, freeze=False)
# loss_fn = ERMLoss(model,torch.nn.CrossEntropyLoss(),{}, subclassed=True)
loss_fn = GDROLossAlt(model,torch.nn.CrossEntropyLoss(),0.5,4)

optimizer = torch.optim.Adam(model.parameters(), lr=0.0005, weight_decay=0.005)#lr=0.001, weight_decay=0.005)
scheduler = ReduceLROnPlateau(optimizer, mode='max', factor=0.2, patience=2, verbose=True)


/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and will be removed in 0.15, "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


#train function

In [147]:
def train_epochs(epochs, train_loader, val_loader, model, loss_fn, optimizer,scheduler, verbose=True):
    for epoch in range(epochs):
        print(f"Epoch {epoch + 1}/{epochs}")
        train(train_loader, model, loss_fn, optimizer, verbose=verbose)
        accuracies = test(val_loader, model, verbose=verbose)
        scheduler.step(accuracies[2])

In [160]:
train_epochs(15,tr_loader,cv_loader,model,loss_fn,optimizer,scheduler)

Epoch 1/15
Average training loss: 0.5596827298402787
Accuracy: 0.7891156462585034 
Accuracy over subgroups: [0.90322581 0.66666667 0.75       1.        ] 
Worst Group Accuracy: 0.6666666666666666
Epoch 2/15
Average training loss: 0.43175351694226266
Accuracy: 0.8095238095238095 
Accuracy over subgroups: [0.82051282 0.96363636 0.46428571 0.84      ] 
Worst Group Accuracy: 0.4642857142857143
Epoch 3/15
Average training loss: 0.2991642463952303
Accuracy: 0.8231292517006803 
Accuracy over subgroups: [0.70967742 0.84615385 0.81818182 0.94444444] 
Worst Group Accuracy: 0.7096774193548387
Epoch 4/15
Average training loss: 0.23262961786240338
Accuracy: 0.8367346938775511 
Accuracy over subgroups: [0.92       0.91935484 0.54054054 1.        ] 
Worst Group Accuracy: 0.5405405405405406
Epoch 5/15
Average training loss: 0.20768878590315581
Accuracy: 0.7551020408163265 
Accuracy over subgroups: [0.84       0.67391304 0.72916667 0.85714286] 
Worst Group Accuracy: 0.6739130434782609
Epoch 00005: redu

In [161]:
test(tst_loader, model, verbose=True)

Accuracy: 0.8704318936877077 
Accuracy over subgroups: [0.8627451  0.86065574 0.80519481 1.        ] 
Worst Group Accuracy: 0.8051948051948052


(0.8704318936877077,
 0.8627450980392157,
 0.860655737704918,
 0.8051948051948052,
 1.0)

In [152]:
next(tst_loader)[0].shape

torch.Size([301, 3, 71, 71])

In [58]:
235-128

107